In [21]:
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
import h5py

In [22]:
# Add Brainweb files have same extension
files = glob.glob('*.rawb')

class Tissue:
    '''
    Tissue class to store MRI data and parameters
    '''

    def __init__(self, volume):
        self.volume = volume
        self.T1 = -1
        self.T2 = -1
        self.T2star = -1
        self.density = -1
        self.susceptibility = -1
        self.name = 'None'
        self.freq = 0.0 

    def __getitem__(self, idx):
        return ((self.volume[idx] + 128) / 255)

    def __setitem__(self, idx, value):
        self.volume[idx] = value

    def export(self, filename):
                
        with h5py.File(filename, 'w') as f:
            f.create_dataset('volume', data=self.volume, compression='gzip')
            f.attrs['T1'] = self.T1
            f.attrs['T2'] = self.T2
            f.attrs['T2star'] = self.T2star
            f.attrs['density'] = self.density
            f.attrs['susceptibility'] = self.susceptibility
            f.attrs['name'] = self.name
            f.attrs['freq'] = self.freq

    def load(self, filename):
    
        with h5py.File(filename, 'r') as f:
            self.volume = np.array(f['volume'])
            self.T1 = f.attrs['T1']
            self.T2 = f.attrs['T2']
            self.T2star = f.attrs['T2star']
            self.density = f.attrs['density']
            self.susceptibility = f.attrs['susceptibility']
            self.name = f.attrs['name']
            self.freq = f.attrs['freq']

    
# Load the data and create the Tissue objects
tissues = []
for file in files:
    data = np.fromfile(file, dtype=np.int8)
    data = data.reshape((362, 434, 362))

    file_prefix = os.path.splitext(file)[0]

    tissue = Tissue(data)
    tissue.name = file_prefix
    
    # https://brainweb.bic.mni.mcgill.ca/brainweb/tissue_mr_parameters.txt
    # J.Z. Bojorquez et al. / Magnetic Resonance Imaging 35 (2017) 69–80 
    # Schneck https://aapm.onlinelibrary.wiley.com/doi/epdf/10.1118/1.597854 
    if 'dura' in file_prefix:
        tissue.T1 = 1000
        tissue.T2 = 100
        tissue.T2star = 50
        tissue.density = 0.6
        tissue.susceptibility = -9.05
    if 'csf' in file_prefix:
        tissue.T1 = 4000
        tissue.T2 = 2000
        tissue.T2star = 2000
        tissue.density = 1.0
        tissue.susceptibility = -9.05
    if 'fat' in file_prefix:
        tissue.T1 = 250
        tissue.T2 = 70
        tissue.T2star = 60
        tissue.density = 1.0
        tissue.susceptibility = -7.79
        tissue.freq = 440 # Hz
    if 'bck' in file_prefix:
        tissue.T1 = 800
        tissue.T2 = 800
        tissue.T2star = 800
        tissue.density = 0.0
        tissue.susceptibility = 0.36
    if 'muscle' in file_prefix:
        tissue.T1 = 900
        tissue.T2 = 50
        tissue.T2star = 40
        tissue.density = 1
        tissue.susceptibility = -9.05
    if 'skin' in file_prefix:
        tissue.T1 = 2569
        tissue.T2 = 329
        tissue.T2star = 58
        tissue.density = 1.0
        tissue.susceptibility = -9.05
    if 'marrow' in file_prefix:
        tissue.T1 = 1000
        tissue.T2 = 50
        tissue.T2star = 40
        tissue.density = 1.0
        tissue.susceptibility = -9.05
    if 'vessels' in file_prefix:
        tissue.T1 = 1600
        tissue.T2 = 80
        tissue.T2star = 70
        tissue.density = 1.
        tissue.susceptibility = -9 #-6.56
    if 'wm' in file_prefix:
        tissue.T1 = 500
        tissue.T2 = 70
        tissue.T2star = 61
        tissue.density = 0.77
        tissue.susceptibility = -9.05
    if 'gm' in file_prefix:
        tissue.T1 = 900
        tissue.T2 = 90
        tissue.T2star = 69
        tissue.density = 0.86
        tissue.susceptibility = -9.05
    if 'skull' in file_prefix:
        tissue.T1 = 300
        tissue.T2 = 1
        tissue.T2star = 1
        tissue.density = 0.1
        tissue.susceptibility = -8.86
  
    print(f'{tissue.name} T1 = {tissue.T1} T2 = {tissue.T2} T2star = {tissue.T2star} density = {tissue.density} susceptibility = {tissue.susceptibility}')
    tissues.append(tissue)  


subject54_muscles_v T1 = 900 T2 = 50 T2star = 40 density = 1 susceptibility = -9.05
subject54_bck_v T1 = 800 T2 = 800 T2star = 800 density = 0.0 susceptibility = 0.36
subject54_marrow_v T1 = 1000 T2 = 50 T2star = 40 density = 1.0 susceptibility = -9.05
subject54_muscles_skin_v T1 = 2569 T2 = 329 T2star = 58 density = 1.0 susceptibility = -9.05
subject54_wm_v T1 = 500 T2 = 70 T2star = 61 density = 0.77 susceptibility = -9.05
subject54_fat_v T1 = 250 T2 = 70 T2star = 60 density = 1.0 susceptibility = -7.79
subject54_fat2_v T1 = 250 T2 = 70 T2star = 60 density = 1.0 susceptibility = -7.79
subject54_vessels T1 = 1600 T2 = 80 T2star = 70 density = 1.0 susceptibility = -9
subject54_dura_v T1 = 1000 T2 = 100 T2star = 50 density = 0.6 susceptibility = -9.05
subject54_gm_v T1 = 900 T2 = 90 T2star = 69 density = 0.86 susceptibility = -9.05
subject54_csf_v T1 = 4000 T2 = 2000 T2star = 2000 density = 1.0 susceptibility = -9.05
subject54_skull_v T1 = 300 T2 = 1 T2star = 1 density = 0.1 susceptibili

In [24]:
# Generate a 3D dipole kernel for forward QSM model
def generate_3d_dipole_kernel(data_shape, voxel_size, b_vec):
    fov = np.array(data_shape) * np.array(voxel_size)

    rz, ry, rx = np.meshgrid(np.arange(-data_shape[0] // 2, data_shape[0] // 2),
                             np.arange(-data_shape[1] // 2, data_shape[1] // 2),
                             np.arange(-data_shape[2] // 2, data_shape[2] // 2), indexing='ij')

    rz, ry, rx = rz / fov[0], ry / fov[1], rx / fov[2]

    sq_dist = rx ** 2 + ry ** 2 + rz ** 2
    sq_dist[sq_dist == 0] = 1e-6
    d2 = ((b_vec[0] * rz + b_vec[1] * ry + b_vec[2] * rx) ** 2) / sq_dist
    kernel = (1 / 3 - d2)

    return kernel

# Apply a forward convolution to the susceptibility maps in k-space
def forward_convolution(chi_sample):
    scaling = np.sqrt(chi_sample.size)
    chi_fft = np.fft.fftn(chi_sample) / scaling
    
    chi_fft_t_kernel = chi_fft * np.fft.fftshift(generate_3d_dipole_kernel(chi_sample.shape, voxel_size=250.25, b_vec=[1, 0, 0]))
   
    tissue_phase = np.fft.ifftn(chi_fft_t_kernel)
    tissue_phase = np.real(tissue_phase * scaling)

    return tissue_phase
  
# Get chi from fuzzy maps
chi = np.zeros(tissues[0].volume.shape)
for tissue in tissues:
    chi += tissue[:,:,:]*tissue.susceptibility
    print(f'{tissue.name} susceptibility: {tissue.susceptibility} max volume: {tissue[:,:,:].max()}')
    
# Simulate the phase maps
phs_tissue_simulated = forward_convolution(chi)

# Estimate the gradient of the phase maps to estimate T2*
grad = np.gradient(phs_tissue_simulated)
grad = np.sqrt( grad[0]**2 + grad[1]**2 + grad[2]**2)
grad = grad / np.mean(grad)

subject54_fat_v susceptibility: -7.79 max volume: 1.0
subject54_csf_v susceptibility: -9.05 max volume: 1.0
subject54_gm_v susceptibility: -9.05 max volume: 1.0
subject54_wm_v susceptibility: -9.05 max volume: 1.0
subject54_dura_v susceptibility: -9.05 max volume: 1.0
subject54_marrow_v susceptibility: -9.05 max volume: 1.0
subject54_muscles_skin_v susceptibility: -9.05 max volume: 1.0
subject54_skull_v susceptibility: -8.86 max volume: 1.0
subject54_vessels susceptibility: -9 max volume: 1.0
subject54_bck_v susceptibility: 0.36 max volume: 1.0
subject54_muscles_v susceptibility: -9.05 max volume: 1.0
subject54_fat2_v susceptibility: -7.79 max volume: 1.0


In [25]:
# For interactive plotting
from ipywidgets import interact, interactive, FloatSlider, IntSlider, ToggleButton
from IPython.display import clear_output, display, HTML

# for plotting modified style for better visualization
import matplotlib.pyplot as plt 
import matplotlib as mpl
mpl.rcParams['lines.linewidth'] = 4
mpl.rcParams['axes.titlesize'] = 16
mpl.rcParams['axes.labelsize'] = 14
mpl.rcParams['xtick.labelsize'] = 12
mpl.rcParams['ytick.labelsize'] = 12
mpl.rcParams['legend.fontsize'] = 12

In [32]:
# Export the Tissue objects to HDF5 files
for tissue in tissues:
    tissue.export(f'{tissue.name}.h5')

# Export gradient and phase
grad = np.iinfo(np.uint16).max*grad / np.max(grad)
with h5py.File('gradient.h5', 'w') as f:
    f.create_dataset('gradient', data=grad.astype(np.uint16), compression='gzip')
    #f.create_dataset('phase', data=phs_tissue_simulated, compression='gzip')   


In [30]:
# Need 
# Tissues
# Gradient

def calc_signal( TE, TR, B0, freq, alpha, M0, T1, T2, T2star, spin_echo, grad_slice):
    # T1, T2, T2star in ms
    # B0 in Hz
    # alpha in degrees
    # M0 in arbitrary units
    # TE, TR in ms

    # convert B0 to rad/s
    B0 = B0 * 2 * np.pi

    # convert alpha to rad
    alpha = alpha * np.pi / 180

    # calculate the longitudinal magnetization based on flip angle
    #Mxy = Mz * np.sin(alpha) * np.exp(-TE / T2star) 
    if spin_echo:
        Mz180 = -M0*(1 - np.exp(-0.5*TE/T1))
        Mz = M0 + (Mz180 - M0) * np.exp(-(TR-0.5*TE) / T1)
        Mxy = Mz * np.exp(-TE / T2)
    else:
        Mz = M0 * (1 - np.exp(-TR / T1)) / (1 - np.cos(alpha) * np.exp(-TR / T1))
        Mxy = Mz * np.sin(alpha) * np.exp(-TE / T2star) * np.exp(-grad_slice*TE/40) * np.exp(-1j*2*np.pi*freq*TE/1000)
    
    return Mxy


def plot_image(TR, TE, flip, slice, spin_echo, noise_level):
    

    #print(f'{TR}, {TE}, {flip}')

    signal = np.zeros((434, 362), dtype=np.complex128)
    for tissue in tissues:
            signal += calc_signal(TE, TR, 0, tissue.freq, flip, tissue[slice]*tissue.density, tissue.T1, tissue.T2, tissue.T2star, spin_echo, grad[slice])

    signal = np.abs( signal + np.random.normal(0, noise_level, signal.shape))

    plt.figure()
    plt.imshow(np.flip(signal), cmap='gray')

    #plt.imshow(np.flip(phs_tissue_simulated[slice]), cmap='gray')

    plt.xticks([])
    plt.yticks([])
    
    if spin_echo:
        plt.title(f' SE:flip={int(flip)},TR={int(TR)},TE={int(TE)} ')
    else:
        plt.title(f'GRE:flip={int(flip)},TR={int(TR)},TE={int(TE)} ')

    plt.colorbar()
    plt.show()



TRslider = FloatSlider(min=2, max=5000, step=1, value=50, description='TR [ms]',continuous_update=True)
TEslider = FloatSlider(min=1, max=50, step=1, value=3, description='TE [ms]',continuous_update=True)
flip_slider = FloatSlider(min=1, max=90, step=1, value=10, description='Flip [deg.]',continuous_update=True)
spin_echo_toggle = ToggleButton(value=False, description='Spin echo', continuous_update=True)

def update_max_TE(change):
    TEslider.max = min(change['new']-1,100.0)

def update_flip_min(change):
    
    if change['new']:
        flip_slider.max = 90
        flip_slider.min = 90
    else:
        flip_slider.max = 90
        flip_slider.min = 1


TRslider.observe(update_max_TE, names='value')
spin_echo_toggle.observe(update_flip_min, names='value')

w = interactive(plot_image, 
                TR=TRslider,
                TE=TEslider,
                flip=flip_slider,
                slice=IntSlider(min=0, max=362, step=1, value=130, description='Slice',continuous_update=True),
                spin_echo=spin_echo_toggle,
                noise_level=FloatSlider(min=0, max=0.1, step=0.001, value=0.01, description='Noise level',continuous_update=True)
                )
display(w)

interactive(children=(FloatSlider(value=50.0, description='TR [ms]', max=5000.0, min=2.0, step=1.0), FloatSlid…